In [1]:
### Run - 1 

import pandas as pd
import numpy as np
import os
import glob

# list_of_files = glob.glob(r'D:\Masterdata\Clean Data/*.csv')
# latest_file = max(list_of_files, key=os.path.getctime)

list_of_files=os.listdir(r"D:\1. Masterdata\Clean Data")
a=[r"D:\1. Masterdata\Clean Data/"+i for i in list_of_files]
latest_file = max(a, key=os.path.getctime)
print(latest_file)

data_all = pd.read_csv(latest_file, sep = ';',converters = {'Phone' : str, 'Order #' : str, 'AWB' : str})

print(data_all['Channel'].unique())

j_conv = pd.read_excel("Settlement Project\Rumus Realisasi.xlsx", sheet_name = 'Paket Brand')
j_conv = j_conv.iloc[:,:5]
j_conv = j_conv.rename(columns = {'SKU' : 'Real SKU', 'SKU.1' : 'SKU Produk'})
j_conv = j_conv[j_conv['Real SKU'].notnull()]
j_conv['SKU Produk'] = j_conv['SKU Produk'].astype(str).str.replace('.0', '',regex = False)

temp_all = data_all.copy()

temp_all['Real SKU'] = temp_all['Real SKU'].astype(str)
j_conv['Real SKU'] = j_conv['Real SKU'].astype(str)

temp_all = temp_all.merge(j_conv, how = 'left', on = 'Real SKU')

paket_j = temp_all[temp_all['SKU Produk'].notnull()]

paket_j['Bundle Name'] = paket_j['Product Name']
paket_j['Product Name'] = paket_j['Name']
paket_j['SKU'] = paket_j['SKU Produk']
paket_j['Qty. Invoiced'] = paket_j['Qty. Invoiced'] * paket_j['Konversi ke UOM'].astype(float)
paket_j['Bundle Flag'] = np.nan

import time
import pandas as pd
import numpy as np
import math

import smtplib 
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
import requests
import os
    
print('sku tatanama')
data_SKU = pd.read_excel(r"D:\1. Masterdata\SKU_File\data_SKU.xlsx")

s = requests.Session()
s.get("https://tatanama.pythonanywhere.com")
s.post("https://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
r = s.get("https://tatanama.pythonanywhere.com/download")

with open(r"D:\1. Masterdata\SKU_File\Master tatanama.xlsx", 'wb') as output:
    output.write(r.content)

if os.path.isfile(r"D:\1. Masterdata\SKU_File\Master tatanama.xlsx") :    
    SKU_append = pd.read_excel(r"D:\1. Masterdata\SKU_File\Master tatanama.xlsx")
    SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
    data_SKU = data_SKU[~data_SKU['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
    data_SKU = data_SKU.append(SKU_append, ignore_index = True, sort = False)
print('sku tatanama done')
    
data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)
data_SKU.loc[data_SKU['Price List NFI'].isin(['-']), 'Price List NFI'] = 0
data_SKU['Price List NFI'] = data_SKU['Price List NFI'].astype(float).astype('int64')
if 'Real SKU_y' in paket_j.columns:
    paket_j = paket_j.drop('Real SKU_y', axis = 1)
paket_j['SKU'] = paket_j['SKU'].astype(str)
paket_j[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU', 'Price List NFI']] = paket_j.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU', 'Price List NFI']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(paket_j.index)[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y', 'Price List NFI_y']]

paket_j['Price List NFI']=paket_j['Price List NFI'].astype(int)
paket_j['Total Net'] = paket_j['Price List NFI'] * paket_j['Qty. Invoiced']

paket_j['Subtotal'] = paket_j['Subtotal'] * paket_j['Total Net']/paket_j.groupby(['Order #', 'Bundle Name'])['Total Net'].transform('sum')
paket_j['Selling Price'] = paket_j['Subtotal']/paket_j['Qty. Invoiced']
paket_j['Total'] = paket_j['Total'] * paket_j['Total Net']/paket_j.groupby(['Order #', 'Bundle Name'])['Total Net'].transform('sum')

paket_j['Seller Discount'] = paket_j['Seller Discount'] * paket_j['Total Net']/paket_j.groupby(['Order #', 'Bundle Name'])['Total Net'].transform('sum')
paket_j['Shipping'] = paket_j['Shipping'] * paket_j['Total Net']/paket_j.groupby(['Order #', 'Bundle Name'])['Total Net'].transform('sum')

paket_j = paket_j.drop(['Nama Paket', 'SKU Produk', 'Name', 'Konversi ke UOM'], axis = 1)
data_all = data_all.append(paket_j, ignore_index = True, sort = False)
indeks = data_all[data_all['Product Name'].isin(paket_j['Bundle Name'])].index.to_list()
data_all['Brand'][indeks] = 'Bundle'

indeks = data_all[data_all['SKU'].astype(str) == '2152051'].index.to_list()
data_all['SKU'][indeks] = '2152051110'
data_all['Price List NFI'][indeks] = data_SKU[data_SKU['SKU'] == '2152051110']['Price List NFI'].values[0]
data_all['Total Net'][indeks] = data_all['Price List NFI'][indeks] * data_all['Qty. Invoiced'][indeks]

indeks = data_all[data_all['Real SKU'].astype(str) == '2152051'].index.to_list()
data_all['Real SKU'][indeks] = '2152051110'
data_all['Price List NFI'][indeks] = data_SKU[data_SKU['SKU'] == '2152051110']['Price List NFI'].values[0]
data_all['Total Net'][indeks] = data_all['Price List NFI'][indeks] * data_all['Qty. Invoiced'][indeks]

data_all['PL Before PPN'] = data_all['Price List NFI']/1.11
data_all['Total Net Before PPN'] = data_all['PL Before PPN'] * data_all['Qty. Invoiced']

indeks = data_all[(data_all['Real SKU'].astype(str).str.contains('^\(B\)')) &
         ~(data_all['Brand'].isin(['Bonus Produk', 'Gimmick']))].index.to_list()
data_all['Brand'][indeks] = 'Bonus Produk'

indeks = data_all[(data_all['Real SKU'].astype(str).str.contains('^\(U\)')) &
         ~(data_all['Brand'].isin(['Bonus Produk', 'Gimmick']))].index.to_list()
data_all['Brand'][indeks] = 'Bonus Produk'

aladin = data_all[
    (data_all['Channel'] == 'Aladin Mall') &
    (data_all['Year'] >= 2021) 
#     (data_all['True datetime'] < '2022-01-27')
]

D:\1. Masterdata\Clean Data/data_all_23 May With Order Online.csv


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3170: DtypeWarning: Columns (12,15,20,24,25,27,29,30,31,32,33,34,35,36,37,39,53,54,57,59,60,66,67,73,74,80,81,87,88,94,95,101,102,108,122,123,134,135,136,153,159,161,167,168,169,172,173,174,175,176,180,181,183,196,197,198,199,200,201) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['L-Men Store Blibli' 'Nutrimart' 'Tokopedia' 'Shopee' 'FBL' 'Blibli'
 'TikTok' 'Lazada' 'JD Indonesia' 'Bukalapak' 'Aladin Mall'
 'Order Online Surabaya' 'Order Online Sumsel' 'Order Online Jateng'
 'Order Online Bali' 'Order Online Medan' 'Order Online Lampung'
 'Order Online Pekanbaru' 'Order Online Jabar' 'Order Online Makassar'
 'Order Online Samarinda' 'Order Online Banjarmasin' 'Orami Shopping'
 'GENERAL' 'GARUDA TIMUR PACIFIC, PT' 'SHOPEE INTERNATIONAL INDONESIA, PT'
 'TANI SUPPLY INDONESIA, PT' 'WICAKSANA OVERSEAS INDONESIA, PT'
 'RITEL BERSAMA NASIONAL, PT' 'WARUNG PINTAR DISTRIBUSI, PT'
 'ENSEVAL PUTERA MEGATRADING TBK, PT' 'TEMPO, PT' 'KREASI TANI LAKSMI, PT'
 'BINTANG ASET INDONESIA, PT' 'CHILIBELI BAGUS INDONESIA, PT'
 'NATA GERAI ERABARU, PT' 'KONEKSI NIAGA SOLUSINDO, PT'
 'PASKOMNAS NIAGA UTAMA, PT' 'RITEL BERSAMA NASIONAL'
 'ENSEVAL PUTERA MEGATRD' 'ASTRO TECHNOLOGIES, PT' 'ECOM RETAIL - ASTRO'
 'ECOM RETAIL - BAYI NINJA' 'ECOM RETAIL - JD ID'
 'ECOM RETAIL - SAYUR BOX

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

sku tatanama
sku tatanama done


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pand

In [3]:
#Run - 2 ubah tanggal 

aladin = data_all[
    (data_all['Channel'] == 'Aladin Mall') &
    (data_all['Year'] > 2021) &
    (data_all['True datetime'] < '2023-05-23')
]

In [4]:
# optional

pd.options.display.max_columns = 150
aladin['True datetime'].max()

'2023-04-26 18:35:00'

In [5]:
del(data_all)

# Selcen Gaada

# WMS

In [6]:
## run - 3
for_wms = aladin.copy()

In [7]:
## run -4

import pandas as pd
import numpy as np
import pypyodbc

conn = pypyodbc.connect('Driver={SQL Server};'
                        'Server=NFI-DB-01;'
                        'Database=gudang;'
                        # 'UID=kelvin.oktavian;'
                        # 'PWD=24nutrifoodindonesia22;'
                         'Trusted_Connection=yes;'
                       )

cursor = conn.cursor()
wms_telkom = pd.read_sql(r"""SELECT t1.id, t1.order_id,t1.status, t1.quantity, t1.product_id, t2.id,  t2.order_number, t2.invoice_number, t2.channel, t2.order_date, t3.id, t3.sku, t3.name
  FROM [gudang].[dbo].[outbound_items] as t1
  left join orders as t2
  on t1.order_id = t2.id
  left join products as t3
  on t1.product_id = t3.id
  """, conn)

In [6]:
## run -4 alternative

wms_telkom=pd.read_excel(r"D:\6. settlement\Code\Settlement Project\Manual\backup wms\data_wms_telkom 21 Apr 2023.xlsx")

In [8]:
## run-5
for_wms['Order #'] = for_wms['Order #'].astype(str).str.replace('-1', '', regex = False)

In [9]:
## run-6
for_wms['Order #'] = for_wms['Order #'].astype(str).str.replace('-2', '', regex = False)

In [10]:
## for backup
wms_telkom_cp = wms_telkom.copy()

In [10]:
## for backup
wms_telkom=wms_telkom_cp.copy()

In [11]:
## run-7

# import pandas as pd
# import numpy as np
# import pypyodbc

# conn = pypyodbc.connect('Driver={SQL Server};'
#                 'Server=NFI-DB-01;'
#                 'Database=gudang;'
#                 'UID=andra.miftah;'
#                 'PWD=24nutrifoodindonesia21;'
#                 )

# cursor = conn.cursor()
# wms_telkom = pd.read_sql(r"""SELECT t1.id, t1.order_id,t1.status, t1.quantity, t1.product_id, t2.id,  t2.order_number, t2.invoice_number, t2.channel, t2.order_date, t3.id, t3.sku, t3.name
#   FROM [gudang].[dbo].[outbound_items] as t1
#   left join orders as t2
#   on t1.order_id = t2.id
#   left join products as t3
#   on t1.product_id = t3.id
#   where t2.channel = 'Aladin Mall'""", conn)

wms_telkom = wms_telkom[wms_telkom['channel'] == 'Aladin Mall']
wms_telkom.loc[wms_telkom['invoice_number'].isnull(),'invoice_number']=wms_telkom['order_number'] #### tambahan timo 27 Juli 2022

# outbound_items = pd.read_csv(r'Backup Gudang 4 Sept 2021 Gudang Baru/outbound_items.csv', sep = ';')
# orders = pd.read_csv(r'Backup Gudang 4 Sept 2021 Gudang Baru/orders.csv', sep = ';')
# products = pd.read_csv(r'Backup Gudang 4 Sept 2021 Gudang Baru/products.csv', sep = ';')

# wms_telkom2 = outbound_items[['id', 'order_id','status', 'quantity', 'product_id']].merge(
#     orders[['id',  'order_number', 'invoice_number', 'channel', 'order_date']], how = 'left', left_on = 'order_id', right_on = 'id'
# ).merge(
#     products[['id', 'sku', 'name']], how = 'left', left_on = 'product_id', right_on = 'id'
# )
# wms_telkom2 = wms_telkom2[wms_telkom2['channel'].isin(['TikTok Commerce'])]
# wms_telkom = wms_telkom[['invoice_number', 'sku', 'status', 'quantity']].drop_duplicates()
# wms_telkom2 = wms_telkom2[['invoice_number', 'sku', 'status', 'quantity']].drop_duplicates()
# wms_telkom = wms_telkom.append(wms_telkom2, ignore_index = True, sort = False)

# wms_telkom = wms_telkom.groupby(['invoice_number', 'sku', 'status'])['quantity'].sum().reset_index()

wms_telkom['WH'] = 'Telkom'
wms_telkom = wms_telkom.rename(columns = {'invoice_number' : 'Order #', 'status':'SO STATUS', 'sku' : 'SKU WH', 'quantity' : 'Warehouse Quantity'})

### tambahan timo nov 2022
if 'STATUS' in wms_telkom.columns:
    wms_telkom = wms_telkom.rename(columns = {'invoice_number' : 'Order #', 'STATUS':'SO STATUS', 'sku' : 'SKU WH', 'quantity' : 'Warehouse Quantity'})

    
for_wms['Order #'] = for_wms['Order #'].astype(str)
wms_telkom['Order #'] = wms_telkom['Order #'].astype(str)

temp_telkom = for_wms[for_wms['Order #'].isin(wms_telkom['Order #'])]

# temp_telkom = temp_telkom.drop(['WH', 'Order #', 'SO STATUS', 'SKU WH', 'Warehouse Quantity'], axis = 1)
temp_telkom['Order #'] = temp_telkom['Order #'].astype(str)
wms_telkom['Order #'] = wms_telkom['Order #'].astype(str)

temp_telkom['Real SKU'] = temp_telkom['Real SKU'].astype(str).str.replace('.0', '', regex = False)
wms_telkom['SKU WH'] = wms_telkom['SKU WH'].astype(str)

wms_telkom = wms_telkom.groupby(['Order #', 'SKU WH', 'SO STATUS', 'WH'], dropna = False)['Warehouse Quantity'].sum().reset_index()

temp_telkom = temp_telkom.merge(wms_telkom.drop_duplicates(['Order #', 'SKU WH']), how = 'left', left_on = ['Order #', 'Real SKU'], right_on = ['Order #', 'SKU WH'])

temp_telkom['Warehouse Quantity'] = temp_telkom['Warehouse Quantity'] * temp_telkom['Qty. Invoiced']/temp_telkom.groupby(['Order #', 'Real Nama Produk'])['Qty. Invoiced'].transform('sum')

temp_telkom_bundle = temp_telkom[temp_telkom['Brand'] == 'Bundle']

temp_buat_bundle = temp_telkom[temp_telkom['Warehouse Quantity'].notnull()].groupby(['Order #', 'Bundle Name']).agg({'WH' : 'first', 'SO STATUS' : 'first', 'Warehouse Quantity' : 'max', 'Qty. Invoiced' : 'max'}).reset_index()
temp_buat_bundle['Warehouse Quantity'] = temp_buat_bundle['Warehouse Quantity']/temp_buat_bundle['Qty. Invoiced']
temp_buat_bundle = temp_buat_bundle.drop('Qty. Invoiced', axis = 1)

temp_telkom_bundle = temp_telkom_bundle.merge(temp_buat_bundle.drop_duplicates(['Order #', 'Bundle Name']), how = 'left', left_on = ['Order #', 'Product Name'], right_on = ['Order #', 'Bundle Name']).set_index(temp_telkom_bundle.index)
temp_telkom_bundle['Warehouse Quantity_y'] = temp_telkom_bundle['Warehouse Quantity_y']*temp_telkom_bundle['Qty. Invoiced']

temp_telkom['SO STATUS'][temp_telkom_bundle.index] = temp_telkom_bundle['SO STATUS_y']
temp_telkom['WH'][temp_telkom_bundle.index] = temp_telkom_bundle['WH_y']
temp_telkom['Warehouse Quantity'][temp_telkom_bundle.index] = temp_telkom_bundle['Warehouse Quantity_y']

for_wms = for_wms[~for_wms['Order #'].isin(wms_telkom['Order #'])]
for_wms = for_wms.append(temp_telkom, ignore_index = True, sort = False)

wms_copy = wms_telkom.copy()
wms_copy['SKU WH'] = wms_copy['SKU WH'].astype(str).str.replace('(E)', '', regex = False)

aladin_wms = for_wms.copy()

null_telkom = aladin_wms[(aladin_wms['WH'].isnull())
            & 
            (aladin_wms['Order #'].isin(wms_telkom['Order #']))]

null_telkom = null_telkom.drop(['WH', 'SO STATUS', 'Warehouse Quantity'], axis = 1)
  
null_telkom['Order #'] = null_telkom['Order #'].astype(str)
wms_copy['Order #'] = wms_copy['Order #'].astype(str)

null_telkom['Real SKU'] = null_telkom['Real SKU'].astype(str).str.replace('.0', '', regex = False)
wms_copy['SKU WH'] = wms_copy['SKU WH'].astype(str).str.replace('.0', '', regex = False)

null_telkom = null_telkom.merge(wms_copy.drop_duplicates(['Order #', 'SKU WH']), how = 'left', left_on = ['Order #', 'Real SKU'], right_on = ['Order #', 'SKU WH']).set_index(null_telkom.index)

null_telkom['Warehouse Quantity'] = null_telkom['Warehouse Quantity'] * null_telkom['Qty. Invoiced']/null_telkom.groupby(['Order #', 'Real Nama Produk'])['Qty. Invoiced'].transform('sum')

null_telkom_bundle = null_telkom[null_telkom['Brand'] == 'Bundle']

temp_buat_bundle = null_telkom[null_telkom['Warehouse Quantity'].notnull()].groupby(['Order #', 'Bundle Name']).agg({'WH' : 'first', 'SO STATUS' : 'first', 'Warehouse Quantity' : 'max', 'Qty. Invoiced' : 'max'}).reset_index()
temp_buat_bundle['Warehouse Quantity'] = temp_buat_bundle['Warehouse Quantity']/temp_buat_bundle['Qty. Invoiced']
temp_buat_bundle = temp_buat_bundle.drop('Qty. Invoiced', axis = 1)

null_telkom_bundle = null_telkom_bundle.merge(temp_buat_bundle.drop_duplicates(['Order #', 'Bundle Name']), how = 'left', left_on = ['Order #', 'Product Name'], right_on = ['Order #', 'Bundle Name']).set_index(null_telkom_bundle.index)
null_telkom_bundle['Warehouse Quantity_y'] = null_telkom_bundle['Warehouse Quantity_y']*null_telkom_bundle['Qty. Invoiced']

null_telkom['SO STATUS'][null_telkom_bundle.index] = null_telkom_bundle['SO STATUS_y']
null_telkom['WH'][null_telkom_bundle.index] = null_telkom_bundle['WH_y']
null_telkom['Warehouse Quantity'][null_telkom_bundle.index] = null_telkom_bundle['Warehouse Quantity_y']

aladin_wms['SO STATUS'][null_telkom.index] = null_telkom['SO STATUS']
aladin_wms['WH'][null_telkom.index] = null_telkom['WH']
aladin_wms['Warehouse Quantity'][null_telkom.index] = null_telkom['Warehouse Quantity'] 

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Se

# Settlement

In [32]:
### run-8

import pandas as pd
import numpy as np
import os

set_aladin = pd.DataFrame()
for i in os.listdir(r'Settlement Project/Aladin Mall/Settlement/'):
    if r'$' not in i:
        temp = pd.read_excel(r'Settlement Project/Aladin Mall/Settlement/' + str(i))
        temp['Source'] = i
        print(i)
        set_aladin = set_aladin.append(temp, ignore_index = True, sort = False)

03. Settlement AladinMall 2023.xlsx
Settlement AladinMall- ok.xlsx
SR Nutrimart - Periode Payment 24 Jan 2022.xlsx
SR Nutrimart - Periode Pembayaran 2 Feb 2022.xlsx
SR Nutrimart - Periode Pembayaran 28 Jan 2022.xlsx
SR Nutrimart - Periode Pembayaran 9 Feb 2022.xlsx


In [33]:
## run - 9
set_aladin = set_aladin[set_aladin['order_ids'].notnull()]

In [34]:
## run -10
set_aladin['order_ids'] = set_aladin['order_ids'].astype(str).str.replace('.0', '', regex = False)
set_aladin['sku'] = set_aladin['sku'].astype(str).str.replace('.0', '', regex = False)

In [35]:
# run -11
set_aladin = set_aladin.drop_duplicates(['order_ids', 'sku'])

In [15]:
### optional
set_aladin[set_aladin.duplicated(['order_ids', 'sku'], keep = 'first')]

,purchase_order_id,purchase_date,order_ids,order_dates,shipping_name,vendor_name,3PL_NAME,qty,status,deliver_date,product_name,product_price,no_abw_or_track_number,jenis_shippment,sku,Voucher Name,voucher Tanggungan Seller,Buttom Price (Product Price - Voucher),Buttom Price Flash sale,Shipping cost per item,Total Price (Qty x Prd),Margin %,Margin Value,OS to Pay,Ongkir To Pay,Total Ammount To be Paid,Paid / Hold / Not Paid,Paid Date,Tanggal Transaksi,Keterangan,Cabang,Jumlah,Cust Name,Unnamed: 33,Tanggal Transaksi.1,Keterangan.1,Cabang.1,Jumlah.1,Cust Name.1,Source,shipping_price,total_cost,status_order,last_date_order,Periode TRX,Unnamed: 22,Unnamed: 23,Total Price (Qty x Prd).1,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 34,Unnamed: 35,method,statement


In [36]:
## run -12 
set_aladin['Comfee Prop'] = set_aladin['Margin Value']
set_aladin['Total Produk Prop'] = set_aladin['Total Ammount To be Paid']

In [37]:
## run - 13
set_aladin = set_aladin[['order_ids', 'sku', 'Comfee Prop', 'Total Produk Prop']]

In [38]:
# run -14

for_wms['Order #'] = for_wms['Order #'].astype(str)
for_wms['Total Net Before PPN'] = for_wms['Total Net']/1.11

In [39]:
## run -15
bundling = for_wms[for_wms['Bundle Name'].notnull()]
real_trx = for_wms[for_wms['Bundle Name'].isnull()]

real_trx['Order #'] = real_trx['Order #'].astype(str)
set_aladin['order_ids'] = set_aladin['order_ids'].astype(str)

real_trx = real_trx.merge(set_aladin, how = 'left', left_on = ['Order #', 'Real SKU'], right_on = ['order_ids', 'sku'])

non_prop = real_trx[real_trx['Brand'].isin(['Bonus Produk', 'Gimmick'])]
real_trx = real_trx[~real_trx['Brand'].isin(['Bonus Produk', 'Gimmick'])]

# real_trx['Total Produk Prop'] = real_trx['Total Produk'].fillna(0) * real_trx['Total Net Before PPN']/real_trx.groupby(['Order #'])['Total Net Before PPN'].transform('sum')
# real_trx['Comfee Prop'] = real_trx['Comfee'].fillna(0) * real_trx['Total Net Before PPN']/real_trx.groupby(['Order #'])['Total Net Before PPN'].transform('sum')

non_prop['Total Produk Prop'] = 0
non_prop['Comfee Prop'] = 0

# display(real_trx[real_trx['Order #']=='1045065239'])#[['Total Produk Prop']]


real_trx = real_trx.append(non_prop, ignore_index = True, sort = False)

bundling['Order #'] = bundling['Order #'].astype(str)

bundling = bundling.merge(real_trx[['Order #', 'Product Name', 'Total Produk Prop', 'Comfee Prop']].drop_duplicates(['Order #', 'Product Name']), how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name'])

non_prop = bundling[bundling['Brand'].isin(['Bonus Produk', 'Gimmick'])]
bundling = bundling[~bundling['Brand'].isin(['Bonus Produk', 'Gimmick'])]
# bundling['Total Produk Prop']=bundling['Total Produk Prop'].astype(float)

bundling['Total Produk Prop'] = bundling['Total Produk Prop'].fillna(0) * bundling['Total Net Before PPN']/bundling.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
bundling['Comfee Prop'] = bundling['Comfee Prop'].fillna(0) * bundling['Total Net Before PPN']/bundling.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')

non_prop['Total Produk Prop'] = 0
non_prop['Comfee Prop'] = 0

bundling = bundling.append(non_prop, ignore_index = True, sort = False)

# real_trx = real_trx.drop(['SKU_y', 'Real SKU_y', 'SKU_y'], axis = 1)
# real_trx = real_trx.rename(columns = {'SKU_x' : 'SKU'})

# bundling = bundling.drop(['SKU_y', 'Real SKU_y', 'SKU_y', 'Product Name_y'], axis = 1)
# bundling = bundling.rename(columns = {'SKU_x' : 'SKU', 'Product Name_x' : 'Product Name'})

total_trx = real_trx.append(bundling, ignore_index = True, sort = False)

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [40]:
# total_trx[total_trx['Order #'].isin(set_aladin['order_ids'])][['Order #', 'Real SKU', 'Product Name', 'Bundle Name', 'Total Produk Prop']]
# total_trx[total_trx['Order #']=='1045067423'][['Order #', 'Real SKU', 'Product Name', 'Bundle Name', 'Total Produk Prop']]

In [41]:
# pd.options.display.max_columns = 150

In [42]:
# real_trx[['Order #', 'Product Name', 'Total Produk Prop', 'Comfee Prop']].drop_duplicates(['Order #', 'Product Name'])

In [43]:
# bundling.merge(real_trx[['Order #', 'Product Name', 'Total Produk Prop', 'Comfee Prop']].drop_duplicates(['Order #', 'Product Name']), how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name'])

In [44]:
### run -16
total_trx['Paid'] = np.nan

indeks = total_trx[total_trx['Order #'].isin(set_aladin['order_ids'])].index.to_list()
total_trx['Paid'][indeks] = 'Paid'

total_trx['Paid'] = total_trx['Paid'].fillna('Unpaid')

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [45]:
# pd.options.display.max_columns = 150

In [46]:
## run -17
total_trx['Product Name'] = total_trx['Product Name'].fillna(total_trx['Product Name_x'])
total_trx = total_trx.drop(['Product Name_x', 'Product Name_y'], axis = 1)

In [47]:
### run -18
total_trx['Total Produk Prop Adj'] = total_trx['Total Produk Prop']/1.11
total_trx['BTL Cost'] = (total_trx['Total Produk Prop Adj']) - total_trx['Total Net Before PPN']

In [48]:
## run -19
total_trx['Total Selcen'] = total_trx['Total']
total_trx['Order Status_y'] = total_trx['Order Status']

In [49]:
## run 20

total_trx_full = total_trx[['Order #',
 'Sales Order ID',
 'AWB',
 'Paid Date',
 'Order date',
 'Week',
 'Date',
 'Month',
 'Quarter',
 'Year',
 'Channel',
 'SKU',
 'Brand',
 'Product Name',
 'Bundle Name',
 'Price List NFI',
 'Qty. Invoiced',
 'Total Net',
 'Sub Brand',
 'Real SKU',
 'Real Nama Produk',
 'Parent Item',
 'Parent SKU',
 'Bundle Flag',
 'Shipping Courier',
 'Store',
 'True datetime',
 'Store Type',
 'Region Group',
 'PL Before PPN',
 'Total Net Before PPN',
 'Category Baru',
 'Exported Parent Item',
 'Total Selcen',
 'Order Status_y',
 'Paid',
 'SO STATUS',
 'WH',
 'Warehouse Quantity',
 'Total Produk Prop',
 'BTL Cost',
 'Total Produk Prop Adj', 
 'Comfee Prop'
 ]]

total_trx_full = total_trx_full.rename(columns = {
    'Total Selcen' : 'Total Seller Centre', 
    'Order Status_y' : 'Order Status Seller Centre',
    'SO STATUS' : 'Status Gudang'
})

In [50]:
### run -21 ganti file bulan lalu ganti var prev_month

import pandas as pd

prev = pd.read_csv(r"Settlement Project\Backup\Realisasi Apr 2023\Final Result\Raw Unpivoted\CSV\Running AladinMall Apr 2023.csv", sep = ';')
prev_month = 'April'
prev = prev.rename(columns = {'Paid' : 'Paid ' + prev_month})
# prev['Qty. Invoiced'] = prev['Qty. Invoiced'].astype(float)
current = total_trx_full.copy()
# current = pd.read_csv("D:\Backup\Backup 26 Mei\Settlement Project Backup\Realisasi Januari 2022\Final Result\Running Nutrimart Jan Jan 2022.csv", sep = ';')

prev['Order #'] = prev['Order #'].astype(str)
prev['Real SKU'] = prev['Real SKU'].astype(str)
current['Order #'] = current['Order #'].astype(str)
current['Real SKU'] = current['Real SKU'].astype(str)

prev['Bundle Name'] = prev['Bundle Name'].astype(str)
indeks = prev[prev['Bundle Name'] == 'nan'].index.to_list()
prev['Bundle Name'][indeks] = np.nan

merged = current.merge(prev[['Order #','Real SKU', 'Paid ' + prev_month, 'Bundle Name']].drop_duplicates(['Order #', 'Real SKU', 'Bundle Name']), how = 'left', on = ['Order #', 'Real SKU', 'Bundle Name'])
import numpy as np

merged['Paid Beda'] = np.nan

indeks = merged[merged['Paid'] != merged['Paid ' + prev_month]].index.to_list()
merged['Paid Beda'][indeks] = 'Beda'

merged['Paid Beda'] = merged['Paid Beda'].fillna('Sama')
# merged[merged['Paid Beda'] == 'Beda'][['Order #','Brand','Real SKU','Month', 'Paid_x', 'Paid_y', 'Paid Beda']]
merged[merged['Paid Beda'] == 'Beda'].groupby(['Year', 'Month', 'Paid', 'Paid ' + prev_month], dropna = False)['Order #'].nunique()

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Year    Month  Paid  Paid April
2023.0  April  Paid  Unpaid        6
                     NaN           6
Name: Order #, dtype: int64

In [51]:
merged = merged.rename(columns = {'Paid' : 'Paid May'})
# merged.to_csv(r'D:\Backup\Backup 26 Mei\Settlement Project Backup/Realisasi April 2022/Final Result/Running AladinMall Apr 22 Rev 1.csv', sep = ';', index = False)

In [52]:
merged.to_csv(r'Settlement Project\Backup\Realisasi May 2023\Final Result\Raw Unpivoted\CSV/Running AladinMall May 2023.csv', sep = ';', index = False)

In [53]:
merged.to_excel(r'Settlement Project\Backup\Realisasi May 2023\Final Result\Raw Unpivoted\Excel/Running AladinMall May 2023.xlsx', index = False)

In [23]:
pricelaz=pd.read_excel("D:\Downloads\pricestock19566export1656468900963_0629-10-15-00.xlsx")

In [26]:
pricelaz['SellerSKU']=pricelaz['SellerSKU'].astype(str)

In [34]:
pricelaz.columns

Index(['Product ID', 'catId', '*Nama Produk', 'currencyCode', 'sku.skuId',
       'Variations Combo', 'Shop SKU', 'status', '*Stock', '*Price',
       'SpecialPrice', 'SpecialPrice Start', 'SpecialPrice End', 'SellerSKU',
       'tr(s-wb-product@md5key)'],
      dtype='object')

In [28]:
priceshopee=pd.read_excel("D:\Downloads\Promo Plan 7-7 Shopee.xlsx")

In [31]:
priceshopee['SKU NFI']=priceshopee['SKU NFI'].astype(str)

In [33]:
priceshopee['Harga Reg']=priceshopee['Harga 7-7']+500

In [ ]:
for item in priceshopee['SKU NFI'].unique():
    print(item)
    harganew=priceshopee[priceshopee['SKU NFI']==item]['Harga Reg'].mean()
    pricelaz.loc[pricelaz['SellerSKU']==item,'SpecialPrice']=harganew
    

In [42]:
pricelaz.to_excel('adjust to 77 shopee.xlsx',index=False)

In [41]:
priceshopee[priceshopee['SKU NFI']=='2104405P2']

,SKU NFI,Product Name,Harga 7-7,Harga Reg
191,2104405P2,Twin Pack - Tropicana Slim Minyak Kanola 946ml...,129900,130400
192,2104405P2,Twin Pack - Tropicana Slim Minyak Kanola 946ml...,129900,130400
